# Emulator: Celerite

This notebook provides a template to build an emulator with [Celerite](https://celerite.readthedocs.io/en/stable/). 

There are many Gaussian process (GP) Python libraries that are commonly used to solve regression problems. The two we used here are [Celerite](https://celerite.readthedocs.io/en/stable/) and [George](https://george.readthedocs.io/en/latest/). Both of these libraries use a kernel-based, Bayesian method of regression. A visual exploration of GPs can be found [A Visual Exploration of Gaussian Processes](https://distill.pub/2019/visual-exploration-gaussian-processes/) and a more complete theory behind GPs can be found for free at [ Rasmussen & Williams (2006)](http://www.gaussianprocess.org/gpml/)

The basic idea behind GPs is to use Gaussian distributions (also referred to as *normal* distributions), defined by a mean vector and a covariance matrix (also called kernel), to predict a function at certain *test* points. Each GP library has their own choise of built-in kernels and the option to build your own. The choice of kernel can make a big difference on the success of the regression and finding the best kernel for your own dataset is a bit of an art. Each GP library also has their own strengths and limitations. In terms of using them as emulators we found that George provides good regression models, is able to build GP regressors in 2 or 3 dimensions, but it takes a long time to emulate. Celerite on the other hand is a very fast way to build regressors but it is limited to 1 dimension and is not very accurate in fitting datasets with multiple inputs. 




#### Index<a name="index"></a>
1. [Import packages](#imports)
2. [Load data](#loadData)
    1. [Load train data](#loadTrainData)
    2. [Load test data](#loadTestData)
3. [Emulator method](#emulator)
    1. [Scale data](#scaleData)
    2. [Train emulator](#trainEmu)
    3. [Plot results](#plotEmu)

## 1. Import packages<a name="imports"></a>


In [ ]:
import pickle

import celerite
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import scipy.optimize as op
import seaborn as sns

from celerite import terms
from matplotlib import pylab
from sklearn.preprocessing import StandardScaler

#### Aesthetic settings

In [ ]:
%matplotlib inline

sns.set(font_scale=1.3, style="ticks")

## 2. Load data<a name="loadData"></a>

Read the training data from a `.pickle` file:

### 2.1. Load training data<a name="loadTrainData"></a>

In [ ]:
path_train = '../data/cosmology_train_1d.pickle'
with open(path_train, 'rb') as input_file:
    data_train = pickle.load(input_file)

In [ ]:
input_train = data_train['input_data']
output_train = data_train['output_data']
number_train = input_train.shape[0]
number_param = input_train.shape[1] - 1
number_outputs = output_train.shape[1] - 1
print("Number of datapoints:", number_train)
print("Number of input parameters:", number_param) # remove the `object_id` column
print("Number of outputs:", number_outputs) # remove the `object_id` column

In [ ]:
extra_train = data_train['extra_input']
r_vals = extra_train['r_vals']

In [ ]:
xs_train = input_train.drop(columns=['object_id'])
ys_train = output_train.drop(columns=['object_id'])

### 2.2. Load test data<a name="loadTrainData"></a> 

In [ ]:
path_test = '../data/cosmology_test_1d.pickle'
with open(path_test, 'rb') as input:
    data_test = pickle.load(input)

In [ ]:
input_test = data_test['input_data']
output_test = data_test['output_data']
number_test = input_test.shape[0]
print("Number of datapoints:", number_test)

In [ ]:
xs_test = input_test.drop(columns=['object_id'])
ys_test = output_test.drop(columns=['object_id'])

## 3. Emulator method<a name="emulator"></a>

### 3.1. Scale data<a name="scaleData"></a>

Let's first scale our input parameters, to make training easier:

In [ ]:
scaler = StandardScaler()
scaler.fit(xs_train)

In [ ]:
xs_train.iloc[:] = scaler.transform(xs_train)
xs_test.iloc[:] = scaler.transform(xs_test)

In [ ]:
y_mean = np.mean(ys_train, axis=0)
ys_train = ys_train/y_mean
ys_test = ys_test/y_mean

### 3.2. Train emulator<a name="trainEmu"></a>

In [ ]:
def fit_gp(kernel, x, y, x_new):
    
    x = x.iloc[:,0]
    x_new = x_new.iloc[:,0]
    
    def neg_log_like(params,y,gp): 
        gp.set_parameter_vector(params)
        loglike = gp.log_likelihood(y)
        return -loglike if np.isfinite(loglike) else 1e25

    def grad_neg_log_like(params, y, gp):
        gp.set_parameter_vector(params)
        return -gp.grad_log_likelihood(y)[1]
    
    
    gp = celerite.GP(kernel, mean=0, fit_mean=False)
    gp.compute(x)
    #print("Initial log-likelihood: {0}".format(gp.log_likelihood(y)))
    
    # Fit for the maximum likelihood parameters
    bounds = gp.get_parameter_bounds()
    results = op.minimize(neg_log_like, gp.get_parameter_vector(), 
                       jac=grad_neg_log_like, args=(y, gp))
    gp.set_parameter_vector(results.x)
    #print("Final log-likelihood: {0}".format(-results.fun))

    # Make the maximum likelihood prediction
    gp_mean, gp_cov = gp.predict(y, x_new, return_var=True)
    std = np.sqrt(gp_cov)
    
    return gp_mean, gp_cov

In [ ]:
number_outputs = np.shape(ys_test)[1]
print(number_outputs)
ys_test_preds = ys_test.copy()
ys_train_0 = ys_train.iloc[:, 0]
for i in np.arange(number_outputs):
    print(i)
    ys_train_i = ys_train.iloc[:, i]
    
    term1 = terms.RealTerm(-1, 8.5)
    term2 = terms.JitterTerm(log_sigma=10)
    term3 = terms.RealTerm(log_a=np.log(np.var(ys_train_i)+5), log_c=-np.log(5.0))
    term4 = terms.RealTerm(np.exp(np.var(ys_train_0)), -2)

    # Try different kernels 
    kernel = term1
    ys_pred, ys_cov = fit_gp(kernel=kernel, x=xs_train, 
                             y=ys_train_i, x_new=xs_test)
    ys_test_preds.iloc[:, i] = ys_pred

Undo all the normalizations.

In [ ]:
ys_test = ys_test*y_mean
ys_test_preds = ys_test_preds*y_mean

### 3.3. Plot results<a name="plotEmu"></a>

We compare our predictions to the truth (choosing a subset for visual clarity). 

In [ ]:
np.random.seed(3)
n_plot = int(0.2*number_test)
idxs = np.random.choice(np.arange(number_test), n_plot)
color_idx = np.linspace(0, 1, n_plot)
colors = np.array([plt.cm.rainbow(c) for c in color_idx])

In [ ]:
plt.figure(figsize=(8,6))
for i in range(n_plot):
    ys_test_i = ys_test.iloc[idxs[i], :]
    ys_pred_i = ys_test_preds.iloc[idxs[i], :]
    if i==0:
        label_test = 'truth'
        label_pred = 'emu_prediction'
    else:
        label_test = None
        label_pred = None
    plt.plot(r_vals, ys_test_i, alpha=0.8, label=label_test, 
             marker='o', markerfacecolor='None', ls='None', color=colors[i])
    plt.plot(r_vals, ys_pred_i, alpha=0.8, label=label_pred, color=colors[i])
plt.xlabel('$r$')
# plt.ylim(-.001,0.015)
plt.ylabel(r'$\xi(r)$')
plt.legend()

We plot the fractional error of all test set statistics:

In [ ]:
color_idx = np.linspace(0, 1, number_test)
colors = np.array([plt.cm.rainbow(c) for c in color_idx])
plt.figure(figsize=(8,6))
frac_errs = np.empty((number_test, number_outputs))
for i in range(number_test):
    ys_test_i = ys_test.iloc[i, :]
    ys_pred_i = ys_test_preds.iloc[i, :]
    frac_err = (ys_pred_i-ys_test_i)/ys_test_i
    frac_errs[i] = frac_err
    plt.plot(r_vals, frac_err, alpha=0.8, color=colors[i])
plt.axhline(0.0, color='k')
plt.xlabel('$r$')
plt.ylabel(r'fractional error')

We show the spread of these fractional errors:

In [ ]:
plt.figure(figsize=(8,6))
for i in range(n_plot):
    ys_test_i = ys_test.iloc[idxs[i], :]
    ys_pred_i = ys_test_preds.iloc[idxs[i], :]
    frac_err = (ys_pred_i-ys_test_i)/ys_test_i
    plt.plot(r_vals, frac_err, alpha=0.8, color=colors[i])
plt.axhline(0.0, color='k')
plt.xlabel('$r$')
plt.ylabel(r'fractional error')

[Go back to top.](#index)